
# How did Tensorflow 1.x look like?

Programs implemented in Tensorflow can be thought of as consisting of two parts:
    - Parts for building up a "computational graph".
    - Parts for the execution of the graph.

Computational graphs consist of Tensorflow operations, where _each "node" represents a mathematical operation_. 

The main purpose of this structure is to allow the high speed execution of the said computations as well as _automatic differentiation_ , meaning the automatic computation of derivatives e.g. in case of a neural network and backpropagation. 

Tensorflow **1.x** was using a _"define and run"_ approach, meaning that the  definition and execution phases happen separately, where other frameworks (like Pytorch and Chainer) utilize a _"define by run"_ approach. The latter is becoming more and more widespread, so Tensorflow also incorporated [Tensorflow Eager Execution](https://www.tensorflow.org/guide/eager), but since the baseline approach was the "define and run", we illustrate Tensorflow usage at first by this.

## Basic ops

In the cell below let us build up a very simple computational graph. All nodes require 0 or more _tensors_ and return _tensors_ themselves. 

The most basic of these is [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant), which expects no input, and returns a value stored in it internally.

We can define two floating point constants as follows:

In [ ]:
#Don't forget to import Tensorflow
import tensorflow as tf

# This is _NOT_ a good practice, but for presentation purposes
# We wish for a clean output, so we suppress TF's warnings
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False


# For the illustration of TF1 behavior, we switch off eager execution, which is by now on by default 
tf.compat.v1.disable_eager_execution()

# as seen below, all ops have a dtype - very much in numpy style
tensor1 = tf.constant(3.0, dtype=tf.float32)
tensor2 = tf.constant(4.0) # also tf.float, implicit
print(tensor1, tensor2)


Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


**Important**

Please notice that by executing the above cell we _don't_ get the result of 3.0 and 4.0, but instead the two atypical variables only give back their "value" if we evaluate (execute) the computational graph. 

For this we need a **Session**, that is an "execution run". The code below opens a Session on a computational resource, eg. a GPU or a CPU and through the "run" method executes the **computational graph** which was statefully stored. (This is important to remember!)

In [ ]:
sess =  tf.compat.v1.Session()
print(sess.run([tensor1, tensor2]))

[3.0, 4.0]


Now we get back the results of 3.0 and 4.0.

By adding new operation nodes to the graph we can build up a more complex graph.

In [ ]:
tensor3 = tf.add(tensor1, tensor2)
print("tensor3:", tensor3)
print("sess.run(tensor3):", sess.run(tensor3))


tensor3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(tensor3): 7.0


This is for the time being not an exceptionally interesting graph, since it always gives back the same result. If we would like to use variable inputs in our computation, we need **Placeholders**. The placeholder can be considered as a promise that we will give it a value some time during the execution of the graph.

In [ ]:
a = tf.compat.v1.placeholder(tf.float32)
b = tf.compat.v1.placeholder(tf.float32)
sum = a + b  # + is a shorthand fo tf.add(), _not_ python + in this case, since it is executed on tf objects
print(sess.run(sum, {a: 3, b: 4.5}))
print(sess.run(sum, {a: [1, 3], b: [2, 4]}))
# Notice, that the execution worked easily for vectors also, so + became a pointwise addition 

7.5
[3. 7.]


In machine learning we would typically like to have a model that can handle arbitrary input - as above, and the graph has to be modifiable, "trainable", so that by changing the parameters it should give different responses to the same inputs. The trainable parameters are represented by **Variables** in Tensorflow which represent the learned parameters of the model. _Before use, Variables have to be initialized_ to some value.

In [ ]:
#Some initial parameters we set up for the "model"
w = tf.Variable([[.3],[.2],[.5],[.27]], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# We define the placeholder that will accept the inputs later on
x = tf.compat.v1.placeholder(tf.float32, shape=(1,4))

# We define a simple transformation akin to a linear model
my_transformation = tf.matmul(x,w)+b
# Bear in mind, this is proper vector multiplication,
# q*r + u would be _pointwise_ multiply, not matrix multiply!

# We initialize the variables, that is, a computational graph gets built in the background
# and the intialization values we provided above gets put into the Variable instances
init = tf.compat.v1.global_variables_initializer()

# We used the previously opened session to execute our init
sess.run(init)
# And finally we feed in some values to execute our model calculation
print(sess.run(my_transformation, {x: [[1., 2., 3., 4.]]}))

[[2.9800003]]


In order to be able to evaluate the model, we create a placeholder for the target value called y, and we write a simple "loss function".

The loss function measures the distance of the output from the target value. This example is a standard linear regression (same as we have used in Scikit), which uses the square of deltas between prediction and target. 


In [ ]:
y = tf.compat.v1.placeholder(tf.float32)
squared_deltas = tf.square(my_transformation - y)
loss = tf.reduce_sum(squared_deltas)
# This is just a funny syntax for sum - albeit potentially by axes...
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum


print(sess.run(loss, {x: [[1, 2, 3, 4]], y: [3.2]}))

0.048399907


## Operations vs tensors in TensorFlow

> Though this be madness, yet there is method in't
> (Polonius in Hamlet)

With simplification:

- tf.Operation objects:  Nodes in the computational graph -- operating on tensors and resulting in tensors.
- tf.Tensor objects: Edges of the computational graph. "A `Tensor` is a symbolic handle to one of the outputs of an
  `Operation`. These represent the values that will flow through the graph. A `tf.Tensor` object represents a partially defined computation that will eventually produce a value."

When we build the computational graphs we _jointly_ create operations and tensors (or more precisely pointers, references to them) in one go:

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
# This is VERY good practice, since the default graph is stateful, so noone knows, what derbis is in there...

const = tf.constant(1, name="Constantine")
print("Const type:", type(const))
print("Const name:", const.name)
print("The Operation for the Const:", type(const.op))
print("The name of the Operation for the Const:", const.op.name)

Const type: <class 'tensorflow.python.framework.ops.Tensor'>
Const name: Constantine:0
The Operation for the Const: <class 'tensorflow.python.framework.ops.Operation'>
The name of the Operation for the Const: Constantine


Please observe, that `name` is an important and _automatically generated_ property of a tensor, that is why we see `Constantine:0` instead of `Constantine` - though we intended to use that. To a quote a Tensorflow developer, this somewhat annoying feature is the result of the fact that:

"...The name of a Tensor is the concatenation of

1. the name of the operation that produced it,
2. a colon (:), and
3. the index of that tensor in the outputs of the operation that produced it.

Therefore the tensor named "foo:2" is the output of the op named "foo" at position 2 (with indices starting from zero).

The naming of `tf.Variable` objects is slightly strange. Every `tf.Variable` contains a mutable tensor object that holds the state of the variable (and a few other tensors). A "Variable" op (which has the name "variable_name" ...) "produces" this mutable tensor each time it is run as its 0th output, so the name of the mutable tensor is "variable_name:0".

Since a `tf.Variable` is mostly indistinguishable from a `tf.Tensor` — in that it can be used in the same places — we took the decision to make variable names resemble tensor names, so the `Variable.name` property returns the name of the mutable tensor."

[source](https://stackoverflow.com/questions/36150834/how-does-tensorflow-name-tensors)

If this is not readily obvious, don't bother, any time you can not fetch a value, remember to add `:0`. ;-)

## Context management

Python has a powerful tool for managing "external resources", like open files, or - by the way - open computational sessions.

Tensorflow's main design pattern typicaly relies on context management to handle the open session, as well as closing it in the end.

### Context managemenat main rule:

**As long as a context manager code block is open, a resource is available.**

The context manager provides opportunity for exposing the resource to the block with a name.

Below an example:

In [ ]:
import tensorflow as tf
del sess
print("Does sess exist?",'sess' in globals())

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    print("Is sess open?",not sess._closed)
    sess.run(init)
    
print("Is sess closed?", sess._closed)


Does sess exist? False
Is sess open? True
Is sess closed? True


Word of warning: Neither `sess.close()`, nor the contex manager trick works when session is not run inside context. This has no practical relavance, but is rather curious...

# More reading:

TensorFlow is changing and evolving extremely rapidly, worth digging into!

Read more under: [A Tour of TensorFlow](https://arxiv.org/pdf/1610.01178.pdf) or in theory: [A Computational Model for TensorFlow](http://delivery.acm.org/10.1145/3090000/3088527/pldiws17mapl-maplmainid2.pdf)

And follow what hapens at: https://www.tensorflow.org/ecosystem/